----
## Technical Report Notebook 2
----
### Tweet Text - Natural Language Processing and EDA

This section of the EDA will analyse the tweet **text** for both events: Brisbane/27th Nov 2014 + Sydney/25th April 2015 and continue cleaning the dataset where appropriate.
The EDA will compare words in the twitter text for both events throughout the 24 hour period and analyse the key words that surround the term "hail".

The notebook will then extend the analysis onto Topic Modelling (using latent Dirichlet allocation) and sentiment analysis.

----

### 2.i. Twitter Text EDA - Data Cleaning

Firstly I will clean and vectorise the full tweet text data prior to analysis...

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

stops = list(stopwords.words("english"))

In [ ]:
# Note these terms were added into stopwords list after the topic modelling (later in the report) as they
# were distorting the groupings
stops.append('youtube')
stops.append('video')
stops.append('via')
stops.append('de')
stops.append('done')
stops.append('amp')
stops.append('like')
stops.append('keep')
stops.append('go')
stops.append('gt')

In [ ]:
# Use function from capstone part2 to clean twitter data

def clean_tweet(tweet):

    #Remove links from tweet
    link_remove = re.sub(r"http\S+", "", tweet)
    
    #Remove non-letters from tweet    
    letters = re.sub("[^a-zA-Z]", " ", link_remove) 
    
    #Convert to lower case, split into individual words
    words = letters.lower().split()                             
    
    #Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    
    #Join the words back into one string separated by space and return the result.
    return (" ".join(meaningful_words))

In [ ]:
# Import the final CSV for use in the Tweet Text Analysis:

hail_data_final = pd.read_csv("./hail_data_final.csv")
hail_data_final.drop('Unnamed: 0',axis=1)
hail_data_final.shape

In [ ]:
# Initialize an empty list to hold the full parsed twitter dataset

clean_hail_data = []

# Loop over each tweet and append the cleanded tweet:

for tweet in hail_data_final["Text"]:
    clean_hail_data.append(clean_tweet(tweet))
    
print(clean_hail_data[:10])
print(len(clean_hail_data))

hail_data_final['tweet_words'] = clean_hail_data

In [ ]:
# Initialize the CountVectorizer

# In this updated version I also investigated 2 word ngrams to pick up any damage related terms
# such as "golf-ball" and "tennis-ball", however too many of the bigrams were obvious duplicates of the single terms
# so I reverted back to single words only.

print ("Vectorising! Please wait...\n")

vectorizer = CountVectorizer(analyzer = "word",tokenizer = None,stop_words = 'english',max_features = 1000) 

# fit_transform() does two functions: First, it fits the model

train_data_features = vectorizer.fit_transform(clean_hail_data)

columns = vectorizer.get_feature_names()

In [ ]:
# Fit cvec and converting the result to a DataFrame.

words_df = pd.DataFrame(vectorizer.transform(clean_hail_data).todense(), columns=vectorizer.get_feature_names())

words_df.fillna(0)

print (words_df.shape)

In [ ]:
# Create the final hail_tweet_text dataframe

hail_tweet_text = pd.concat([hail_data_final, words_df], axis=1)
hail_tweet_text.shape

In [ ]:
# Print top-20 words found in full dataset:

hail_tweet_text.ix[:,26:-1].sum().nlargest(20)

#### Why is the term "Gemini" so prevalent?
The term gemini is appearing an unusually high number of times...  Why?

In [ ]:
# Filter out words relating to 'Gemini'

hail_tweet_text_gemini = hail_tweet_text[hail_tweet_text['gemini']==1]
hail_tweet_text_gemini.ix[:,26:-1].sum().nlargest(20)

**Conclusion:** After investigation, it appears this term is appearing with 3 key words (storm, talking and today). After investigation it appears it is related to horoscopes although the source is not Twittascope, which was identified in Capstone part-2. This is clearly unreliable data and would distort the use of the word storm in modelling (which could be a critical predictor of hail), so we will remove it.

In [ ]:
# Damage frequency plot (count vs time) for both events

sns.factorplot(x="posted_hour", y="damage", hue="Cat_ID", data=hail_tweet_text,
                   size=12,aspect=2, kind="bar",ci=None)
plt.title('Hail Tweets: "Damage" frequency over time for Both Events',size=35)
plt.xlabel('Hour of Day (GMT, +9 hours for Aus local time)')
plt.ylabel('Tweet Frequency')

---
#### Key Words - Event Comparison

_Analysing the key words that surround the term "hail" for both of the events._

In this analysis I will only look at the twitter data that was posted during the actual hours of the hail storm, from Hour-6 through to Hour-10 (this is a conservative range).

In [ ]:
# Filter data into actual storm hours

hail_tt_storm_hours = hail_tweet_text[(hail_tweet_text['posted_hour'] >=6) &\
                                        (hail_tweet_text['posted_hour'] <=10) &\
                                        (hail_tweet_text['hail'] > 0)]

print ("Storm hours data shape:",hail_tt_storm_hours.shape)

print (hail_tt_storm_hours.ix[:,26:-1].sum().nlargest(21))

# Create data of top 20 terms (exlcluding hail)

df = hail_tt_storm_hours.groupby(['Cat_ID'])[['storm','rain','weather'
                                        ,'hit','severe','golf','ball','lightning','thunder','damage','storms'
                                        ,'crazy','snow','australia','sized','news']].sum()
df = df.transpose()
df.columns

In [ ]:
# Plot a comparison chart for the key words used in conjuction with the word 'hail' during of the hailstorm hours:

sns.set(font_scale=1)
fig = plt.figure(figsize=(16,12)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twiny() # Create another axes that shares the same x-axis as ax.

df[144].plot(kind='barh', color='red', ax=ax, fontsize=12,legend=True, position=1)
df[154].plot(kind='barh', color='blue', ax=ax2, fontsize=12, legend=True,position=0)

ax.set_ylabel('Hail key term',size=12)
ax.set_xlabel('Term count over hail period',size=12)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Correlation plot of key words used with hail

hail_tt_storm_hours_terms = hail_tt_storm_hours[['storm','rain','weather','hit','severe','golf','ball','lightning','thunder','damage','storms','crazy','snow','australia','sized','news']]

hail_tt_storm_hours_corr = hail_tt_storm_hours_terms.corr()

f, ax = plt.subplots(figsize=(12, 12))

ax = sns.heatmap(data=hail_tt_storm_hours_corr, square=True, ax=ax)
plt.title('"Hail" Tweets: Correlation plot of key words',size=16)
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14, rotation=45)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14, rotation=0)
plt.show()

In [ ]:
# Finally I will experiment with another visualisation method - a wordcloud.
# This is a package created by amueller on Github: https://github.com/amueller/word_cloud

from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS

d = path.dirname('./images/')

# read the mask image
cloud_mask = np.array(Image.open(path.join(d, "hail.png")))

stopwords = set(STOPWORDS)

text = "".join(hail_tt_storm_hours['tweet_words'])

wc = WordCloud(width=800, height=400,scale=16, background_color="white", max_words=1000, mask=cloud_mask,collocations=False).generate(text)

# show
plt.figure(figsize=(18, 16))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")

#### Key hail words insight: 

Storm is the most prevalent word but the two interesting terms used more in the Brisbane event (Cat_ID 144) were 'golf' and 'ball'. As mentioned in previous capstone deliveries, this term is a key indicator of damaging hail. We know already that the Brisbane event was much costlier from an insurance loss perspective, the prevalence of this term in this event is an **excellent indicator of severity!**

---

### Geographic Analysis of Tweet Data

This analysis looks at the geographical distribution of the twitter data over the two 24 hour time periods. 

Some key limitations of this analysis include:
- The coordinate data was minimal in the original data as it relies on twitter users activating location services on their twitter apps to store coordinate data. 
- Ideally the actual tweet location would be used from the location services, however as a proxy, the registered location of the user was geocoded externally (using Google's Geocoder API) and appended to the twitter data.
- Only 2234 records successfully geocoded (1.7% of total records).
- Various geocoding resolutions resulted, ranging from city/suburb to country level, so the accuracy of the location is inconsistent.

Despite these limitations, plotting the (known) registered locations of twitter users gives a reasonable geospatial representation of the global tweets over the two sample days.

This analysis uses Folium to create an interactive map.

In [ ]:
# create dataframe of tweets with coordinates (and hail only)
hail_tweet_text_geo = pd.read_csv("./hail_tweet_text_geo.csv")
print (len(hail_tweet_text_geo))
hail_tweet_text_geo = hail_tweet_text_geo[hail_tweet_text_geo['longitude']>=-180]
# hail_tweet_text_geo = hail_tweet_text_geo[hail_tweet_text_geo['hail']>=0]
                                                                         
latitude = hail_tweet_text_geo['latitude']
longitude = hail_tweet_text_geo['longitude']
print (len(latitude))

import folium
from folium import plugins
import pandas as pd

WORLD_COORDINATES = (0, 0)

# create empty map zoomed in on WORLD

map1 = folium.Map(location=WORLD_COORDINATES, zoom_start=2)

map1.add_child(plugins.HeatMap(zip(latitude, longitude), radius = 10))

map1

#### Heat Map of All Tweets (over both 24-hour date samples):

![map all tweets](images/map_all_tweets.png)

#### Heat Map of Hail Tweets

![map hail tweets](images/map_hail_tweets.png)

_Note: Using geographic analysis, we can see there is a large concentration in tweets in Texas, US. After investigating this cluster, we can conclude another hail event took place in San Antonio on the 25th April 2015. This event did not seem to be significant and little evidence of this location is picked up in the actual tweet text data._

#### Heat Map of Earthquake Tweets

![map eq tweets](images/map_eq_tweets.png)

## 2.iii. LDA - Topic Modelling

I will now investigate the different groupings of words within the dataset. Latent Dirichlet Allocation will help me discover the hidden semantic clusters in the twitter data within subsets relating to different criteria.

The groups I will investigate include:
      - The Full Dataset
      - Brisbane Hail Data (Cat_ID = 144)
      - Brisbane Hail Data (Cat_ID = 144) in the known hail storm timeframe (6 - 10 hours)
      - Sydney Hail Data (Cat_ID = 154)
      - Sydney Hail Data (Cat_ID = 154) in the known hail storm timeframe (6 - 10 hours)
  
Firstly I will run through each of the LDA processes for the full dataset. Then I will create an LDA function to parse my remaining critera separately..

In [ ]:
# Remove all records with the term gemini:

hail_tweet_text = hail_tweet_text[hail_tweet_text['gemini']==0]

### 2.ii. Bag-of-Words Analysis

In [ ]:
# Print top-20 word occurrences for full dataset:

hail_tweet_text.ix[:,26:-1].sum().nlargest(20)

In [ ]:
# Correlation plot of top-20 words from full dataset:

hail_tweet_text_top20 = hail_tweet_text[['storm' ,'damage','thunder','lightning','flood','nepal',
                                         'hail','earthquake','rain','brisbane','pm','new','quake','hits',
                                         'big','warning' ,'news','snow','weather','game','posted_hour']]

corr_tt20 = hail_tweet_text_top20.corr()

f, ax = plt.subplots(figsize=(12, 12))

ax = sns.heatmap(data=corr_tt20, square=True, ax=ax)
plt.title('"Hail" Tweets: Correlation plot of key words',size=16)
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14, rotation=45)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14, rotation=0)
plt.show()

#### Word Correlation Discussion: _Earthquake!?_

The most significant correlation found is with an interesting new set of words introducued with the secondary dataset on 25-April-2015...

The highest correlated words are: **quake, hits, big and Nepal...** We can now deduce that the twitter data also picked up another major natural catastrophe on this date: the Nepal Earthquake!

The additional discovery of the earthquake event provides validation of the searching method, showing that this analysis can not only predict hail but also other natural disasters. This increases the scope of the project as well as the potential applications.

_I will now analyse the word counts in greater detail by investigating the distribution of word counts..._

---


#### _Word Count Distributions_

In [ ]:
# plotting the count distrubtion of all words in the Twitter data, then filter to greater levels.

hist_counts = pd.Series(words_df.ix[:,26:-1].sum(),index=words_df.columns)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16,8))
    
hist_counts.plot(kind="hist", bins=20, ax=axes[0], title="Histogram - All",color='green')
    
# Filtering all words with a count > 1,000
hist_counts[hist_counts > 1000].plot(kind="hist", bins=20, ax=axes[1], title="Histogram - Counts > 1,000",color='orange')
    
# Filtering all words with a count > 10,000
hist_counts[hist_counts > 10000].plot(kind="hist", bins=20, ax=axes[2], title="Histogram - Counts > 10,000",color='red')


These histograms indicate a strong positive skew of word counts, as expected. When we filter counts above 1,000 and 10,000 we observe groupings in greater detail between 10,000 and 30,000, which implies some words may have been tweeted together often.

Our key investiagtion is into hail, so now I will look specifically at this term and how it differs between datasets...

---

#### Hail Frequency Distribution Over Time

Remember, the two datasets used are from two 24 hour periods where know hail events occurred...

- Cat_ID 144 = Brisbane Hail on 27th Nov 2014
- Cat_ID 154 = Sydney Hail on 25th April 2015

I will now compare the key words in the twitter text for both events through each day's duration. First of all, I will review the key word **'hail'**.

In [ ]:
# Frequency plot of the use of the term 'hail' over time:

sns.set(font_scale=2.5)
sns.factorplot(x="posted_hour", y="hail", hue="Cat_ID", data=hail_tweet_text,
                   size=12,aspect=2, kind="bar",palette="bright",ci=None)
plt.title('Hail Tweets: "Hail" frequency over time for Both Events',size=35)
plt.xlabel('Hour of Day (GMT, +9 hours for Aus local time)')
plt.ylabel('Tweet Frequency')
plt.show()

Key insights:
1. Significant peaks at the same same 6-10 hours - this is in line with the known hail event activity time during the two days:
  - Brisbane Hail (144) on 27th Nov 14 occurred after 3pm
  - Sydney Hail on 25th April 15 occurred at a similar time. _Hail events are known to impact the East Coast of Australia later in the day in the summer months_
- Brisbane event: what occurring at 16/17 hours? First I will check another key term...

In [ ]:
# Plot to check the occurrence of 'hailstorm' for the two events:

plt.figure(figsize=(12,8))
sns.factorplot(x="posted_hour", y="hailstorm", hue="Cat_ID", data=hail_tweet_text,
                   size=12,aspect=2, kind="bar",palette="bright",ci=None)
plt.title('Hail Tweets: "hailstorm" frequency over time for Both Events',size=35)
plt.xlabel('Hour of Day (GMT, +9 hours for Aus local time)')
plt.ylabel('Tweet Frequency')

**Comment:** The 'hailstorm' frequency does not provide any further explanation... What about other words used during this hour?

In [ ]:
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Setup vectorisation

vectorizer = CountVectorizer(stop_words=stops,max_features = 1000)
X = vectorizer.fit_transform(hail_tweet_text['tweet_words'])

# Count the terms in the vectorised vocabulary

from collections import Counter
dict(Counter(vectorizer.vocabulary_).most_common(5))

In [ ]:
# create a dataframe of words

docs = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())

#Setup dictionary for gensim

vocab = {v: k for k, v in vectorizer.vocabulary_.items()}

dict(Counter(vocab).most_common(5))

In [ ]:
# Count the frequency of words

frequency = defaultdict(int)

for text in hail_tweet_text['tweet_words']:
    for token in text.split():
        frequency[token] += 1
        
dict(Counter(frequency).most_common(5))

# Remove single words and stopwords

texts = [[token for token in text.split() if frequency[token] > 1 and token not in stops]
          for text in hail_tweet_text['tweet_words']]

In [ ]:
# Create gensim dictionary object
dictionary = corpora.Dictionary(texts)

dict(Counter(dictionary).most_common(5))

In [ ]:
# Create corpus matrix
corpus = [dictionary.doc2bow(text) for text in texts]
corpus[0:5]

In [ ]:
# Perform LDA modelling

lda = models.LdaModel(
    matutils.Sparse2Corpus(X, documents_columns=False),
    num_topics  =  3, # found to be optimal
    passes      =  5, # passes reduced due to runtime
    id2word     =  vocab)

# Detection of opportunity:
lda.print_topics(num_topics=3, num_words=10)

#### Initial Results:

For the initial topic modelling I selected three grouping of 10 words. Generally the words collected effectively group into known events during the two selected time periods:

- Group 0: **Storm** - A general clustering of terms related to storms: lightning, thunder, time, day, today, etc.
- Group 1: **The Nepal Earthquake** - As observed in the words EDA, the Nepal Earthquake is another major catastrophe that has been identified in the 25th April data. Here we observe _injuries, magnitude, causing, hits_ and other key words, however some other meterological terms slip in too.
- Group 2: **The [Sydney] hail event** - location information, damage, storm and it's characteristics clearly identify this group.

These topics classify the data well and if smaller groups of words were used, the topics would likely be more definitive.

In [ ]:
# Create a correlation plot function to review our topic modelling groups:

def corr_plot(df):
    pref_corr = df.corr()
    mask = np.zeros_like(pref_corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    
    fig, ax = plt.subplots(figsize=(8,7))

    ax = sns.heatmap(pref_corr, mask=mask)

    ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=12, rotation=45)
    ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=12, rotation=0)

    plt.show()

In [ ]:
# Note this group (and subsequent groups) may differ to notebook results because of variation in running the model

hail_group = hail_tweet_text[['storm','flood','hail','rain','sydneystorm',
                                          'sydney','pm','severe','weather','warning']]
corr_plot(hail_group)

**Discussion:** 

Of the terms collected in the "Sydney Hail" group, very few have any significant correlations with each other. 'Sydney' - 'hail', and 'pm' - 'warning', are the only two significant correlations.

I will now analyse groups to see if hail storm topics become more pronounced...

In [ ]:
# Create a topic modelling function to use for all groups I want to investigate

def topic_modelling(twitter_data):
    
    vectorizer = CountVectorizer(stop_words=stops,max_features = 1000)
    X = vectorizer.fit_transform(twitter_data)
    
    vectorizer.get_feature_names()
    
    docs = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())

    vocab = {v: k for k, v in vectorizer.vocabulary_.items()}
    
    frequency = defaultdict(int)

    for text in twitter_data:
        for token in text.split():
            frequency[token] += 1
        
    texts = [[token for token in text.split() if frequency[token] > 1 and token not in stops]
          for text in twitter_data]

    dictionary = corpora.Dictionary(texts)
    
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    lda = models.LdaModel(
    matutils.Sparse2Corpus(X, documents_columns=False),
    num_topics  =  5,
    passes      =  5,
    id2word     =  vocab)

    return lda.print_topics(num_topics=5, num_words=5)

### Brisbane Hail Topic Modelling

#### A. Full Day Analysis

In [ ]:
hail_tweet_text_bris = hail_tweet_text[hail_tweet_text['Cat_ID']==144]
print (hail_tweet_text_bris.shape)

topic_modelling(hail_tweet_text_bris['tweet_words'])

In [ ]:
# Correlation plot:

bris_group1 = hail_tweet_text_bris[['storm','power','damage','clean','hail',]]
corr_plot(bris_group1)

#### B. Brisbane Hail Storm Hours Analysis

In [ ]:
# Brisbane Event data (27th Nov 15); Known hours of hailstorm (Hour 6 to 10)

hail_tweet_text_bris_storm = hail_tweet_text_bris[(hail_tweet_text_bris['posted_hour'] >=6) &\
                                            (hail_tweet_text_bris['posted_hour'] <=10)]

topic_modelling(hail_tweet_text_bris_storm['tweet_words'])

In [ ]:
bris_group2 = hail_tweet_text_bris_storm[['damage','lightning','storm','flood','iphone']]
corr_plot(bris_group2)

### Sydney Hail Topic Modelling

#### A. Full Day Analysis

In [ ]:
hail_tweet_text_syd = hail_tweet_text[hail_tweet_text['Cat_ID']==154]
hail_tweet_text_syd.shape

In [ ]:
# Sydney Hail data (25th April 15)

topic_modelling(hail_tweet_text_syd['tweet_words'])

In [ ]:
syd_group1 = hail_tweet_text_syd[['storm','hail','sydneystorm','thunder','lightning']]
corr_plot(syd_group1)

In [ ]:
# Sydney Hail data (25th April 15); Known hours of hailstorm (Hour 6 to 10)

hail_tweet_text_syd_storm = hail_tweet_text_syd[(hail_tweet_text_syd['posted_hour'] >=6) &\
                                            (hail_tweet_text_syd['posted_hour'] <=10)]

topic_modelling(hail_tweet_text_syd_storm['tweet_words'])

**NRL "purplepride"** is a hashtag for the NRL team Melbourne Storm (they lost Manly Sea Eagles on 25th April 2015!) They did not _win_ and were not _heroes_, but they did play a _game_ - the words are typical to sport sentiment.

In [ ]:
syd_group2 = hail_tweet_text_syd_storm[['storm','damage','flood','time','warning']]
corr_plot(syd_group2)

**Insights of Grouped Topic Modelling:**

The topic modelling groups (the two hail events and their storm durations) improve the recognition of terms likely to be tweeted during hail events (e.g.: meteorology and damage)
No major difference in types of words produced when comparing the Brisbane Hail to the Sydney Hail event, apart from a few terms relating to 'damage' and 'power' in the Brisbane event, and the correlation plots show little improvement between variables than previously observed.
Additional popular events were picked up through this process, including a basketball game between the Indiana Spurs and San Antonia Pacers and Thanksgiving.

## 2.iv. Sentiment Analysis:



In [ ]:
# Using textblob package to perform sentiment scoring

from textblob import TextBlob, Word

def detect_sentiment(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
# create a new DataFrame column for sentiment
hail_tweet_text['sentiment'] = hail_tweet_text.Text.apply(detect_sentiment)

In [ ]:
# plot of sentiment distribtuion for all, hail and earthquake tweets

hail_tweets = hail_tweet_text.loc[hail_tweet_text['hail'] >= 1]
eq_tweets = hail_tweet_text.loc[hail_tweet_text['earthquake'] >= 1]

fig, ax = plt.subplots(figsize=(16,10))
sns.distplot(hail_tweet_text['sentiment'],hist=False,kde_kws={"lw": 1.5, "alpha": 0.6, "label": "all data","color": "grey"})
sns.distplot(hail_tweets['sentiment'],hist=False,kde_kws={"lw": 2, "alpha": 0.8, "label": "hail tweets","color": "b"})
sns.distplot(eq_tweets['sentiment'],hist=False,kde_kws={"lw": 1.5, "alpha": 0.8, "label": "earthquake tweets","color": "r"})
ax.set_xlabel('sentiment score', fontsize=16)
ax.set_ylabel('frequency', fontsize=16)
plt.title("Tweet Sentiment Score by Event Type")

In [ ]:
# plot of log-frequency sentiment distributions of all, earthquake and hail tweets

hail_tweets = hail_tweet_text.loc[hail_tweet_text['hail'] >= 1]
eq_tweets = hail_tweet_text.loc[hail_tweet_text['earthquake'] >= 1]

fig, ax = plt.subplots(figsize=(16,10))
sns.distplot(hail_tweet_text['sentiment'],hist=False,kde_kws={"lw": 4, "alpha": 0.3,"label": "all data","color": "grey"})
sns.distplot(hail_tweets['sentiment'],hist=False,kde_kws={"lw": 1.5, "alpha": 0.7, "label": "hail tweets", "color": "b"})
sns.distplot(eq_tweets['sentiment'],hist=False,kde_kws={"lw": 1.5, "alpha": 0.7, "label": "earthquake tweets","color": "r"})
ax.set_xlabel('sentiment score', fontsize=16)
ax.set_ylabel('frequency', fontsize=16)
plt.title("Tweet Sentiment Score by Event Type (Log Frequency)")
plt.yscale('log')

In [ ]:
# wordcloud of negative sentiment scores for hail

# creating negative sentiment tweet variable for hail tweets
hail_tweets_neg_sent = hail_tweets[hail_tweets['sentiment']<=-0.8]

d = path.dirname('./images/')

# read the mask image
cloud_mask = np.array(Image.open(path.join(d, "cloud.jpg")))

stopwords = set(STOPWORDS)

text = "".join(hail_tweets_neg_sent['tweet_words'])

wc = WordCloud(width=800, height=400,scale=16, background_color="white", max_words=1000, 
               collocations=False).generate(text)
# generate word cloud

# display word cloud
plt.figure(figsize=(18, 16))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")

**Sentiment Analysis Discussion**

The sentiment analysis suggests that at more negative scores, the relative frequency of earthquake and (lesser so) hail tweets are greater than all-tweets combined. The inverse is seen at positive sentiment scores. The above wordcloud provides insight into some of the colourful language used in the negative hail tweets.

Interestingly there appears to be a higher frequency of positive tweets for the earthquake event than for the hail events.this could be people tweeting to record their safety and generally, in very severe natural catastrophes, people tend to tweet very emotive language.

In [ ]:
# Investigation for top 20 words used at hour 17 within the Brisbane/Nov 2014 hail data:

hail_tweet_text_bris = hail_tweet_text[hail_tweet_text['Cat_ID']==144]

hail_tweet_text_bris17 = hail_tweet_text[hail_tweet_text['posted_hour']==17]
print (hail_tweet_text_bris17.ix[:,26:-1].sum().nlargest(20))

In [ ]:
# Plot of source type at hour 17 within the Brisbane/Nov 2014 hail data:

fig = plt.figure(figsize=(12,8))
hail_tweet_text_bris17.source.value_counts().nlargest(10).plot(kind='bar',color='red')
plt.tick_params(labelsize=15)

 #### Brisbane Hail - Hour 17 Discussion?

Inconclusive results:
- Nothing conclusive to indicate the additional signature at hour 17 - nothing is observed from other words and source information...
- It is unclear from this analysis why there is an additional signature later in the day. Possibly international recognition of the event was causing the term to be tweeted?

### "Damage" frequency plot for Both Events: